In [1]:
from langchain_groq import ChatGroq

In [4]:
llm = ChatGroq(
    temperature=0,
    groq_api_key="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",
    model="llama-3.1-70b-versatile",
)

In [5]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-42862?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Apply for Retail Associate II

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsRetail StoresRetail Associate IIKualalumpur, SelangorBecome a Part of the NIKE, Inc. TeamNIKE, Inc. does mo

In [6]:
from langchain_core.prompts import PromptTemplate

promt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTIONS:
    The scraped text is from the career's page of a website,
    your job is to extract the job posting and return them in JSON format containing following keys: 'role', 'experience', 'skills' and 'description'
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE)
"""
)

chain_extract = promt_extract | llm
res = chain_extract.invoke(input={'page_data': page_data})
print(res.content)

```json
{
  "role": "Retail Associate II",
  "experience": "One or more years of customer service and/or retail experience preferred",
  "skills": [
    "Basic math functions, including addition, subtraction, multiplication, and division",
    "Effective communication in verbal and written English",
    "Ability to work effectively with others in a team-oriented environment",
    "Excellent customer service skills",
    "Ability to accomplish multiple tasks in a fast-paced environment",
    "Physical requirements include the ability to twist, bend, squat, reach, climb a ladder and stand for extended periods of time"
  ],
  "description": "As a Nike Direct Athlete, you provide world-class service to the consumer using the four service principles: KNOW, EQUIP, ENGAGE, and INSPIRE. You will demonstrate enthusiasm and eagerness to learn the fundamentals of all store Athlete roles, including but not limited to Service, Visual Presentation, and Stockroom. You will deliver the best possible s

In [13]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Retail Associate II',
 'experience': 'One or more years of customer service and/or retail experience preferred',
 'skills': ['Basic math functions, including addition, subtraction, multiplication, and division',
  'Effective communication in verbal and written English',
  'Ability to work effectively with others in a team-oriented environment',
  'Excellent customer service skills',
  'Ability to accomplish multiple tasks in a fast-paced environment',
  'Physical requirements include the ability to twist, bend, squat, reach, climb a ladder and stand for extended periods of time'],
 'description': 'As a Nike Direct Athlete, you provide world-class service to the consumer using the four service principles: KNOW, EQUIP, ENGAGE, and INSPIRE. You will demonstrate enthusiasm and eagerness to learn the fundamentals of all store Athlete roles, including but not limited to Service, Visual Presentation, and Stockroom. You will deliver the best possible service and attention to all cons

In [8]:
import pandas as pd
df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [10]:
import chromadb
import uuid

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())]
                       )

In [15]:
# links = collection.query(query_texts=["experience in python", "experience in react"], n_results=2).get('metadatas', [])
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/kotlin-backend-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/python-portfolio'}]]

In [16]:
job = json_res
job['skills']

['Basic math functions, including addition, subtraction, multiplication, and division',
 'Effective communication in verbal and written English',
 'Ability to work effectively with others in a team-oriented environment',
 'Excellent customer service skills',
 'Ability to accomplish multiple tasks in a fast-paced environment',
 'Physical requirements include the ability to twist, bend, squat, reach, climb a ladder and stand for extended periods of time']

In [17]:
prompt_email = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION:
    {job_description}

    ### INSRTRUCTION:
    You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.

    ### EMAIL (NO PREAMBLE):
    """
)
chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Enhance Your Retail Operations with AtliQ's Expertise

Dear Hiring Manager,

I came across the job description for a Retail Associate II at Nike, and I was impressed by the emphasis on providing world-class service to consumers. As a Business Development Executive at AtliQ, I believe our company can help Nike streamline its retail operations and enhance the overall customer experience.

At AtliQ, we specialize in developing tailored solutions that cater to the unique needs of our clients. Our expertise in software development, AI, and automation can help Nike optimize its retail operations, reduce costs, and increase efficiency. Our team has a proven track record of delivering high-quality solutions that meet the specific requirements of our clients.

I'd like to highlight a few examples of our work that are relevant to Nike's retail operations:

* Our Magento portfolio (https://example.com/magento-portfolio) showcases our expertise in developing e-commerce solutions that can 